# FUA simplification with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-12-06T20:38:37.497716-05:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.30.0

Compiler    : Clang 17.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib
import time

import cityseer
import geopandas

from core import utils

%watermark -w
%watermark -iv

Watermark: 2.5.0

cityseer : 4.17.2
geopandas: 1.0.1
core     : 0.1.dev181+g858455e



-------------

Within `cityseer==4.17.2` there a dedicated helper function – [`cityseer.tools.io._auto_clean_network()`](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L250)) that combines the elements of the simplification routine. The current sequence within the function (at the time of this writing) is as follows:
1. [initial deduplication](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L259C5-L260)
2. [exclusions & removal](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L261-L329)
3. [removal of dangling nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L330-L331)
4. [splitting opposing geometries](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L340)
   1. This is done with 4 sets of params
5. [consolidate nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L357) 
   1. This is done with 4 sets of params
6. [removal of degree-2 nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L367)
7. [snap gapped endings](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L369)
8. [splitting opposing geometries](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L388)
9. [removal of dangling nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L416)
10. [splitting opposing geometries](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L420)
    1. This is done [twice by default](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L492)
11. [consolidate nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L448)
    1. This is done [twice by default](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L492)
12. [removal of degree-2 nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L477)
13. [merging parallel edges](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L478)
14. [ironing out edges](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L479)
15. [removal of dangling nodes](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L481) 

The current [Manual Cleaning Guide](https://benchmark-urbanism.github.io/cityseer-examples/examples/graph_cleaning.html#manual-cleaning)* mimics above with some keywords that are defaults when run from within [`cityseer.tools.io.osm_graph_from_poly()`](https://github.com/benchmark-urbanism/cityseer-api/blob/4cd1a1b582ae0d53b013e34fa59403cba71712c3/pysrc/cityseer/tools/io.py#L635).

We adapt this sequence as faithfully as possible below in `run_cityseer()`, hopefully following the spirit of `cityseer`-style simplification, while introducing as little bias as possible from our end. We will not include any reliance on OSM tags or special treatment as such, to emphasize the non-use of those in `neatnet`.

**\*** *A curated version of this can be found in the ***Manual cleaning*** section [here](cityseer_4-17-2_Graph-Cleaning-Cityseer-Guide.pdf)*.

-------------

In [3]:
def run_cityseer(city: str) -> geopandas.GeoDataFrame:
    """Run the ``cityseer`` sequence of simplification.

    Parameters
    ----------
    city : str
        City name. See ``utils.city_fua.keys()``.

    Returns
    -------
    geopandas.GeoDataFrame
        Resultant simplified network edges – only geometry column.
        Will be written to ``.parquet``.
    """
    edges = utils.read_original(city)
    graph = cityseer.tools.io.nx_from_generic_geopandas(edges)

    # 1. initial deduplication
    graph = cityseer.tools.graphs.nx_deduplicate_edges(
        graph, dissolve_distance=20, max_ang_diff=20
    )

    # 2. exclusions & removal
    # -- we skip this

    # 3. removal of dangling nodes
    graph = cityseer.tools.graphs.nx_remove_dangling_nodes(
        graph, despine=0, remove_disconnected=100
    )

    # 4. splitting opposing geometries
    graph = cityseer.tools.graphs.nx_split_opposing_geoms(
        graph, squash_nodes=True, centroid_by_itx=True
    )

    # 5. consolidate nodes
    graph = cityseer.tools.graphs.nx_consolidate_nodes(
        graph, crawl=False, centroid_by_itx=True
    )

    # 6. removal of degree-2 nodes
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)

    # 7. snap gapped endings
    graph = cityseer.tools.graphs.nx_snap_gapped_endings(graph, buffer_dist=20)

    # 8. splitting opposing geometries
    graph = cityseer.tools.graphs.nx_split_opposing_geoms(
        graph,
        buffer_dist=20,
        min_node_degree=1,
        max_node_degree=1,
        squash_nodes=False,
    )

    # 9. removal of dangling nodes
    graph = cityseer.tools.graphs.nx_remove_dangling_nodes(graph, despine=40)

    # 10. & 11. splitting opposing geometries & consolidate nodes
    max_angle = 120
    for dist in (5, 10):
        graph = cityseer.tools.graphs.nx_split_opposing_geoms(
            graph,
            buffer_dist=dist,
            squash_nodes=True,
            centroid_by_itx=True,
            simplify_by_max_angle=max_angle,
        )
        graph = cityseer.tools.graphs.nx_consolidate_nodes(
            graph,
            buffer_dist=dist,
            crawl=True,
            centroid_by_itx=True,
            simplify_by_max_angle=max_angle,
        )

    # 12. removal of degree-2 nodes
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)

    # 13. merging parallel edges
    graph = cityseer.tools.graphs.nx_merge_parallel_edges(
        graph, merge_edges_by_midline=True, contains_buffer_dist=50
    )

    # 14. ironing out edges
    graph = cityseer.tools.graphs.nx_iron_edges(
        graph, min_self_loop_length=100, max_foot_tunnel_length=100
    )

    # 15.
    graph = cityseer.tools.graphs.nx_remove_dangling_nodes(graph, despine=25)

    return cityseer.tools.io.geopandas_from_nx(graph, crs=edges.crs).rename_geometry(
        "geometry"
    )[["geometry"]]

In [4]:
utils.city_fua

{'Aleppo': 1133,
 'Auckland': 869,
 'Bucaramanga': 4617,
 'Douala': 809,
 'Liège': 1656,
 'Salt Lake City': 4881,
 'Wuhan': 8989}

In [5]:
output_base = pathlib.Path("..", "..", "data")

In [6]:
global_start = time.time()

for city, fua in utils.city_fua.items():
    print("===================================================================")
    print(f"Starting {city}")
    print("-------------------------")
    print("\n\n")

    fua_start = time.time()
    simplified = run_cityseer(city)
    output_fua = output_base / pathlib.Path(f"{fua}", "cityseer", f"{fua}.parquet")
    simplified.to_parquet(output_fua)
    fua_end = time.time()
    fua_duration = round((fua_end - fua_start) / 60.0, 3)

    print(f"\t{city} completed in: {fua_duration} minutes")
    print("\n\n\n\n")

print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")
print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")

global_end = time.time()
global_duration = round((global_end - global_start) / 60.0, 3)

print(f"Total runtime: {global_duration} minutes")

Starting Aleppo
-------------------------






INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.

INFO:cityseer.tools.util:Creating edges STR tree.


INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.graphs:Removing dangling nodes.
INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.graphs:Consolidating nodes.

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Snapping gapped endings.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:citysee

	Aleppo completed in: 0.381 minutes





Starting Auckland
-------------------------






INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.

INFO:cityseer.tools.util:Creating edges STR tree.


INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.graphs:Removing dangling nodes.
INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.graphs:Consolidating nodes.

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Snapping gapped endings.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:citysee

	Auckland completed in: 0.199 minutes





Starting Bucaramanga
-------------------------





...

	Liège completed in: 0.271 minutes





Starting Salt Lake City
-------------------------






INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.

INFO:cityseer.tools.util:Creating edges STR tree.


INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.graphs:Removing dangling nodes.
INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.graphs:Consolidating nodes.

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Snapping gapped endings.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:citysee

	Salt Lake City completed in: 0.182 minutes





Starting Wuhan
-------------------------






INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.

INFO:cityseer.tools.util:Creating edges STR tree.


INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.graphs:Removing dangling nodes.
INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.graphs:Consolidating nodes.

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.

INFO:cityseer.tools.graphs:Removing filler nodes.

INFO:cityseer.tools.util:Creating nodes STR tree

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Snapping gapped endings.

INFO:cityseer.tools.util:Creating edges STR tree.

INFO:cityseer.tools.graphs:Splitting opposing edges.

INFO:citysee

	Wuhan completed in: 0.387 minutes





~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Total runtime: 2.018 minutes
